In [2]:
import pandas as pd
import os
# import wandb
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, matthews_corrcoef
from transformers import (
    AutoModel,
    AutoTokenizer,
    # DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
# from datasets import Dataset
from accelerate import Accelerator
# from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
# import pickle
from saveAndLoad import *
# import gc

In [3]:
checkpoint = 'facebook/esm2_t30_150M_UR50D'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).to('cuda')
model.eval()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 640, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1026, 640, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-29): 30 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=640, out_features=640, bias=True)
            (key): Linear(in_features=640, out_features=640, bias=True)
            (value): Linear(in_features=640, out_features=640, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=640, out_features=640, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
  

In [4]:
seqs = pickleLoad('../../aa/canonical_mut.pkl')
refs = pickleLoad('../../aa/canonical_ref.pkl')

loading data from ../../aa/canonical_mut.pkl
loading data from ../../aa/canonical_ref.pkl


In [10]:
mutSeqSet = set(seqs)
len(mutSeqSet)

656995

In [11]:
refSeqSet = set(refs)
len(refSeqSet)

1433

In [5]:
with torch.no_grad():
    # idx = 569032
    idx = 0
    print(len(seqs[idx]))
    inputs = tokenizer(seqs[idx])
    ids = torch.tensor([inputs['input_ids']]).to('cuda')
    print(ids)
    att = torch.tensor([inputs['attention_mask']]).to('cuda')
    output = model(ids, attention_mask=att)
    print(len(ids))
output['last_hidden_state'][0][0].shape

189
tensor([[ 0, 20, 11,  9, 19, 15,  4,  7,  7,  7,  6,  5, 23,  6,  7,  6, 15,  8,
          5,  4, 11, 12, 16,  4, 12, 16, 17, 21, 18,  7, 13,  9, 19, 13, 14, 11,
         12,  9, 13,  8, 19, 10, 15, 16,  7,  7, 12, 13,  6,  9, 11, 23,  4,  4,
         13, 12,  4, 13, 11,  5,  6, 16,  9,  9, 19,  8,  5, 20, 10, 13, 16, 19,
         20, 10, 11,  6,  9,  6, 18,  4, 23,  7, 18,  5, 12, 17, 17, 11, 15,  8,
         18,  9, 13, 12, 21, 21, 19, 10,  9, 16, 12, 15, 10,  7, 15, 13,  8,  9,
         13,  7, 14, 20,  7,  4,  7,  6, 17, 15, 23, 13,  4, 14,  8, 10, 11,  7,
         13, 11, 15, 16,  5, 16, 13,  4,  5, 10,  8, 19,  6, 12, 14, 18, 12,  9,
         11,  8,  5, 15, 11, 10, 16, 10,  7,  9, 13,  5, 18, 19, 11,  4,  7, 10,
          9, 12, 10, 16, 19, 10,  4, 15, 15, 12,  8, 15,  9,  9, 15, 11, 14,  6,
         23,  7, 15, 12, 15, 15, 23, 12, 12, 20,  2]], device='cuda:0')
1


torch.Size([640])

In [6]:
mean_pool = lambda x: np.mean(x, 0)
ref_idx_map = pickleLoad('../../aa/idxMap_canonical_mut_to_ref.pkl')
idx = 569032
with torch.no_grad():
    print(len(seqs[idx]))
    inputs = tokenizer(seqs[idx])
    ids = torch.tensor([inputs['input_ids']]).to('cuda')
    att = torch.tensor([inputs['attention_mask']]).to('cuda')
    output0 = model(ids, attention_mask=att)
    output0 = output0['last_hidden_state'].detach().to('cpu').numpy()[0][0]
    # output0 = mean_pool(output0['last_hidden_state'].detach().to('cpu').numpy()[0])

with torch.no_grad():
    idx = ref_idx_map[idx]
    print(len(refs[idx]))
    inputs = tokenizer(refs[idx])
    ids = torch.tensor([inputs['input_ids']]).to('cuda')
    att = torch.tensor([inputs['attention_mask']]).to('cuda')
    output1 = model(ids, attention_mask=att)
    output1 = output1['last_hidden_state'].detach().to('cpu').numpy()[0][0]
    # output1 = mean_pool(output1['last_hidden_state'].detach().to('cpu').numpy()[0])
    print(output1.shape)

print(F.cosine_similarity(torch.tensor(output0).unsqueeze(0),torch.tensor(output1).unsqueeze(0)))

for i,j in zip(output0[:50],output1[:50]):
    print(i,j)

loading data from ../../aa/idxMap_canonical_mut_to_ref.pkl
393
393
(640,)
tensor([0.9999])
0.0077029243 0.0074210595
0.16423267 0.16538818
-0.00051427097 -0.0024153013
0.13874112 0.13559115
-0.24334085 -0.24609078
0.060635354 0.06216118
-0.07986653 -0.077466644
-0.03283909 -0.036811206
-0.3034183 -0.31447315
0.22328214 0.22736904
-0.013858367 -0.015000811
-0.007388341 -0.0074028615
0.1693635 0.16988131
-0.049729064 -0.049305752
0.09429717 0.098850325
-0.025461812 -0.03338955
-0.04144263 -0.037696328
0.23645744 0.23481226
-0.07297695 -0.07374679
-0.15342648 -0.15537073
0.0696569 0.070213035
0.16385235 0.16399106
0.025911558 0.02362929
-0.020788431 -0.023715986
-0.09340263 -0.09518414
-0.15968508 -0.15582308
0.22046402 0.22101621
-0.04719679 -0.052559443
-0.07085347 -0.06972665
-0.44314387 -0.44232017
-0.092319585 -0.08857939
0.08035669 0.07604904
-0.0022764953 -0.00041502813
-0.11165842 -0.10991501
-0.21107888 -0.21400487
-0.0751576 -0.0729533
-0.17121246 -0.16935562
-0.08213998 -0.0840

In [36]:
output['last_hidden_state'].shape

torch.Size([1, 14511, 640])

In [37]:
output['pooler_output'].shape

torch.Size([1, 640])

In [29]:
type(output['pooler_output'].detach().to('cpu').numpy()[0][0])

numpy.float32

In [15]:
max=5
n=0
longest = 0
longest_i = 0
lens = set()
for ni,i in enumerate(seqs):
    if i is None: continue
    lens.add(len(i))
    if len(i)>longest:
        longest = len(i)
        longest_i = ni
    if 900<len(i)<1000:
        if n<max:
            print(ni, len(i))
            n+=1
print('\n',longest,longest_i,'\n')
print(sorted(list(lens))[-15:])


9 976
54 928
57 976
59 976
83 976

 14511 568862 

[5809, 5883, 5889, 5890, 8789, 8794, 8795, 8796, 8797, 8798, 14506, 14507, 14508, 14509, 14511]


In [12]:
emb_mut = np.load('../../aa/canonical_mut_embeddings_esm2.npy')
emb_ref = np.load('../../aa/canonical_ref_embeddings_esm2.npy')

In [13]:
emb_mut[0][:5]

array([-0.07785489,  0.14835379,  0.14564785, -0.13385764, -0.33384255],
      dtype=float32)

In [14]:
emb_ref[0][:5]

array([0.13804878, 0.09532265, 0.0614852 , 0.18461367, 0.14487804],
      dtype=float32)

In [24]:
import pandas as pd
import os
# import wandb
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, matthews_corrcoef
from transformers import (
    AutoModel,
    AutoTokenizer,
    # DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from datasets import Dataset
from accelerate import Accelerator
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import pickle
from saveAndLoad import *
from tqdm import tqdm

muts = pickleLoad('../../aa/canonical_mut.pkl')
refs = pickleLoad('../../aa/canonical_ref.pkl')
data = pickleLoad('../../data_processing/consolidated_data.pkl')
tumors = pickleLoad('../../aa/tumors.pkl') 
        #0:concat_ref
        #1:concat_mut
        #2:iso_ref
        #3:iso_mut
        #4:canonical_ref [115] is None
        #5:canonical_mut [39] is None
        #6:data
mut2ref = pickleLoad('../../aa/idxMap_canonical_mut_to_ref.pkl')

def diff_indices(shorter, longer):
    # Ensure we treat the longer string as 'mutated' for uniform handling of insertions/deletions
    if shorter==longer: return None

    # Find the starting index where the two strings differ
    start = next((i for i, (o, m) in enumerate(zip(shorter, longer)) if o != m), len(shorter))

    # Calculate the length difference 
    len_diff = len(longer) - len(shorter)

    if len_diff == 0:
        # Find the end index of the replacement by comparing substrings starting from 'start'
        end = start + next((i for i, (o, m) in enumerate(zip(shorter[start:], longer[start:])) if o == m), len(shorter) - start)
        return start, end

    # Return the range of indices as 'insertion' or 'deletion'
    return start, start + len_diff

device = 'cuda:1'
checkpoint = 'facebook/esm2_t30_150M_UR50D'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)
model.to(device)
model.eval()

mut_seqs = []
ref_seqs = []

for seq in muts: 
    if seq is None: 
        mut_seqs.append('')
        continue
    seq = seq.split('*')[0]
    mut_seqs.append(seq)

for seq in refs: 
    if seq is None: 
        ref_seqs.append('')
        continue
    seq = seq.split('*')[0]
    ref_seqs.append(seq)

mut_embeddings = np.load('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_average_embeddings_esm2.npy')
print('getting embeddings')
load_val = lambda x: torch.tensor([x]).to(device)

all_seqs = []
count = 0
for i,mut in tqdm(enumerate(mut_seqs), total = len(mut_seqs)):
    ref = ref_seqs[mut2ref[i]]
    if mut == '' or ref == '': 
        all_seqs.append(('',0,0,False))
        continue
    if mut == ref: 
        all_seqs.append(('',0,0,False))
        count+=1
        continue
    deletion = len(ref) > len(mut)
    seq = ref if deletion else mut
    (shorter,longer) = (mut,ref) if deletion else (ref,mut)
    start,end = diff_indices(shorter,longer)
    all_seqs.append((seq,start,end,deletion))
print(count, 'silent mutations')

for i,(seq,start,end,deletion) in tqdm(enumerate(all_seqs), total = len(all_seqs)):
    if all(mut_embeddings[i]==0): 
        if seq=='': continue
        mut_len = end-start
        with torch.no_grad():
            inputs = tokenizer(seq)
            ids = load_val(inputs['input_ids'])
            att = load_val(inputs['attention_mask'])
            output = model(ids, attention_mask=att)
            embedding = output['last_hidden_state'][0][start:end].detach().to('cpu').numpy()
            if mut_len>1: embedding = np.mean(embedding, axis = 0, keepdims = True)
            if deletion: embedding = -embedding
            mut_embeddings[i] = embedding
# np.save('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_average_embeddings_esm2.npy',mut_embeddings)



loading data from ../../aa/canonical_mut.pkl
loading data from ../../aa/canonical_ref.pkl
loading data from ../../data_processing/consolidated_data.pkl
loading data from ../../aa/tumors.pkl
loading data from ../../aa/idxMap_canonical_mut_to_ref.pkl


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.weight', 'esm.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


getting embeddings


  1%|          | 3312/656995 [00:00<00:24, 26911.51it/s]


TypeError: cannot unpack non-iterable NoneType object

In [56]:
##DIFF INDICES TESTS

# Test cases
print(diff_indices('ACGT', 'ACGTTT'))  
print(diff_indices('ACGT', 'ACGG'))  
print(diff_indices('ACGT', 'ACGT'))  
print(diff_indices('ACCGT','ACCCGT'))
print(diff_indices('TACCCGT','TTACCCGT'))
print(diff_indices('TACCCGT','GTACCCGT'))


[4, 5]
[3]
None
[3]
[1]
[0]


In [32]:
def diff_indices(shorter, longer):
    # Ensure we treat the longer string as 'mutated' for uniform handling of insertions/deletions
    if shorter==longer: return None

    # Find the starting index where the two strings differ
    start = next((i for i, (o, m) in enumerate(zip(shorter, longer)) if o != m), len(shorter))

    # Calculate the length difference 
    len_diff = len(longer) - len(shorter)

    if len_diff == 0:
        # Find the end index of the replacement by comparing substrings starting from 'start'
        end = start + next((i for i, (o, m) in enumerate(zip(shorter[start:], longer[start:])) if o == m), len(shorter) - start)
        return start, end

    # Return the range of indices as 'insertion' or 'deletion'
    return start, start + len_diff

def test(ref,mut):
    deletion = len(ref) > len(mut)
    seq = ref if deletion else mut
    (shorter,longer) = (mut,ref) if deletion else (ref,mut)
    start,end = diff_indices(shorter,longer)
    x = '-referece_seq' if deletion else 'mutated_seq'
    print(f'ref: {ref}\nmut: {mut}\n-->  {seq[start:end]} ({x})\n')
    return start

In [33]:
print('SUBSTITUTION')
test(ref = 'ABCDE', mut = 'AYYDE')

print('IN FRAME INSERTION')
test(ref = 'ABCDE', mut = 'AYYBCDE')

print('IN FRAME DELETION')
test(ref = 'ABCDE', mut = 'BCDE')

print('OUT OF FRAME INSERTION')
test(ref = 'ABCDE', mut = 'AYYXLDE')

print('OUT OF FRAME INSERTION CAUSING STOP')
test(ref = 'ABCDE', mut = 'AYYC')

print('OUT OF FRAME DELETION')
test(ref = 'ABCDE', mut = 'AXZE')

print('OUT OF FRAME DELETION CAUSING STOP')
test(ref = 'ABCDE', mut = 'ABX')


SUBSTITUTION
ref: ABCDE
mut: AYYDE
-->  YY (mutated_seq)

IN FRAME INSERTION
ref: ABCDE
mut: AYYBCDE
-->  YY (mutated_seq)

IN FRAME DELETION
ref: ABCDE
mut: BCDE
-->  A (-referece_seq)

OUT OF FRAME INSERTION
ref: ABCDE
mut: AYYXLDE
-->  YY (mutated_seq)

OUT OF FRAME INSERTION CAUSING STOP
ref: ABCDE
mut: AYYC
-->  B (-referece_seq)

OUT OF FRAME DELETION
ref: ABCDE
mut: AXZE
-->  B (-referece_seq)

OUT OF FRAME DELETION CAUSING STOP
ref: ABCDE
mut: ABX
-->  CD (-referece_seq)



2

In [35]:
tumors = pickleLoad('../../aa/tumors.pkl')
tumors

loading data from ../../aa/tumors.pkl


{'GENIE-JHU-00006-00185': [[0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 1, 1],
  [2, 2, 2, 2, 2, 2, 2],
  [3, 3, 3, 3, 3, 3, 3],
  [4, 4, 4, 4, 4, 4, 4],
  [5, 5, 5, 5, 5, 5, 5],
  [6, 6, 6, 6, 6, 6, 6],
  [6, 7, 6, 7, 6, 7, 7],
  [5, 8, 5, 8, 5, 8, 8],
  [7, 9, 7, 9, 7, 9, 9],
  [8, 10, 8, 10, 8, 10, 10],
  [9, 11, 9, 11, 9, 11, 11],
  [10, 12, 10, 12, 10, 12, 12],
  [0, 13, 0, 13, 0, 13, 13],
  [11, 14, 11, 14, 11, 14, 14],
  [11, 15, 11, 15, 11, 15, 15],
  [12, 16, 12, 16, 12, 16, 16]],
 'GENIE-JHU-00007-00187': [[0, 17, 0, 17, 0, 17, 17],
  [8, 18, 8, 18, 8, 18, 18]],
 'GENIE-JHU-00024-00426': [[0, 19, 0, 19, 0, 19, 19],
  [3, 20, 3, 20, 3, 20, 20],
  [9, 21, 9, 21, 9, 21, 21],
  [13, 22, 13, 22, 13, 22, 22]],
 'GENIE-JHU-00026-01223': [[0, 23, 0, 23, 0, 23, 23],
  [8, 24, 8, 24, 8, 24, 24]],
 'GENIE-JHU-00032-00469': [[0, 25, 0, 25, 0, 25, 25],
  [8, 26, 8, 26, 8, 26, 26]],
 'GENIE-JHU-00048-00113': [[2, 27, 2, 27, 2, 27, 27]],
 'GENIE-JHU-00054-01116': [[2, 28, 2, 28, 2, 28, 28],
  [

# FAIR-ESM

In [7]:
import torch
from esm import pretrained
from saveAndLoad import *
import torch.nn.functional as F
import inspect

seqs = pickleLoad('../../aa/canonical_mut.pkl')
refs = pickleLoad('../../aa/canonical_ref.pkl')

loading data from ../../aa/canonical_mut.pkl
loading data from ../../aa/canonical_ref.pkl


In [8]:
model,alphabet = pretrained.esm2_t30_150M_UR50D()
batch_converter = alphabet.get_batch_converter()
print(model.__class__)
print(inspect.getsourcefile(model.__class__))

print(len(seqs[0]))
seqcopy = seqs[:1]
seqcopy.append(seqcopy[0][:10] + '<mask>' + seqcopy[0][11:])
seqcopy.append(seqcopy[0][:10] + 'L' + seqcopy[0][11:])
seqcopy.append(seqcopy[0][:10] + '' + seqcopy[0][11:])

desc = ['position 10 A (wt)','position 10 masked', 'position 10 L (mut)', 'position 10 deletion']

batch_labels, batch_strs, batch_tokens = batch_converter([(i,j) for i,j in enumerate(seqcopy)])
with torch.no_grad():
    output = model(batch_tokens, repr_layers=[model.num_layers])
output['representations'] #[30].shape

AttributeError: module 'esm.pretrained' has no attribute 'esm2_t30_150M_UR50D'

In [146]:
t_to_s = {}
for i,j in zip(batch_tokens[0][1:],batch_strs[0]):
    t_to_s[i.item()] = j
t_to_s

{20: 'M',
 11: 'T',
 9: 'E',
 19: 'Y',
 15: 'K',
 4: 'L',
 7: 'V',
 6: 'G',
 5: 'A',
 23: 'C',
 8: 'S',
 12: 'I',
 16: 'Q',
 17: 'N',
 21: 'H',
 18: 'F',
 13: 'D',
 14: 'P',
 10: 'R'}

In [149]:
def print_outputs(t):
    for i in range(len(output['logits'])):
        print(f'\ntoken embedding {t-1}\n{desc[i]}')
        print(batch_tokens[i][t])
        for nj,j in enumerate(F.softmax(output['logits'][i][t]).tolist()):
            print(f'{nj}:\t {j:.5f}')

In [150]:
print(f'position 10 wt is {batch_tokens[0][11]}')
print_outputs(11)

position 10 wt is 5

token embedding 10
position 10 A (wt)
tensor(5)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00044
5:	 0.93648
6:	 0.01118
7:	 0.00096
8:	 0.03458
9:	 0.00053
10:	 0.00033
11:	 0.00430
12:	 0.00035
13:	 0.00189
14:	 0.00059
15:	 0.00033
16:	 0.00050
17:	 0.00410
18:	 0.00043
19:	 0.00032
20:	 0.00033
21:	 0.00039
22:	 0.00007
23:	 0.00190
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 10
position 10 masked
tensor(32)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00345
5:	 0.34081
6:	 0.14795
7:	 0.01065
8:	 0.32412
9:	 0.00917
10:	 0.00407
11:	 0.03971
12:	 0.00354
13:	 0.01941
14:	 0.00388
15:	 0.00454
16:	 0.00571
17:	 0.05265
18:	 0.00333
19:	 0.00321
20:	 0.00325
21:	 0.00334
22:	 0.00052
23:	 0.01671
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 10
position 10 L (mut)
ten

/tmp/ipykernel_3261046/1125080389.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for nj,j in enumerate(F.softmax(output['logits'][i][t]).tolist()):


In [135]:
print(f'position 100 wt is {batch_tokens[0][100]}')
print_outputs(100)

position 100 wt is 12

token embedding 100
position 11 A (wt)
tensor(12)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00203
5:	 0.00018
6:	 0.00000
7:	 0.00676
8:	 0.00001
9:	 0.00000
10:	 0.00000
11:	 0.00013
12:	 0.99031
13:	 0.00000
14:	 0.00000
15:	 0.00000
16:	 0.00000
17:	 0.00001
18:	 0.00005
19:	 0.00000
20:	 0.00047
21:	 0.00000
22:	 0.00000
23:	 0.00002
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 100
position 11 masked
tensor(12)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00206
5:	 0.00018
6:	 0.00000
7:	 0.00658
8:	 0.00001
9:	 0.00000
10:	 0.00000
11:	 0.00013
12:	 0.99047
13:	 0.00000
14:	 0.00000
15:	 0.00000
16:	 0.00000
17:	 0.00001
18:	 0.00005
19:	 0.00000
20:	 0.00048
21:	 0.00000
22:	 0.00000
23:	 0.00002
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 100
position 11 L (mu

/tmp/ipykernel_3261046/1767336173.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for nj,j in enumerate(F.softmax(output['logits'][i][t]).tolist()):


In [ ]:
print(f'position 9 wt is {batch_tokens[0][10]}')
print_outputs(10)

position 10 wt is 6

token embedding 10
position 11 A (wt)
tensor(6)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00000
5:	 0.00001
6:	 0.99980
7:	 0.00001
8:	 0.00003
9:	 0.00001
10:	 0.00005
11:	 0.00000
12:	 0.00000
13:	 0.00002
14:	 0.00000
15:	 0.00000
16:	 0.00000
17:	 0.00000
18:	 0.00000
19:	 0.00000
20:	 0.00000
21:	 0.00000
22:	 0.00002
23:	 0.00001
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 10
position 11 masked
tensor(6)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00000
5:	 0.00002
6:	 0.99976
7:	 0.00002
8:	 0.00003
9:	 0.00002
10:	 0.00006
11:	 0.00000
12:	 0.00000
13:	 0.00002
14:	 0.00000
15:	 0.00000
16:	 0.00000
17:	 0.00000
18:	 0.00000
19:	 0.00000
20:	 0.00000
21:	 0.00000
22:	 0.00003
23:	 0.00002
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 10
position 11 L (mut)
tens

/tmp/ipykernel_3261046/1767336173.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for nj,j in enumerate(F.softmax(output['logits'][i][t]).tolist()):


In [151]:
print(f'position 11 wt is {batch_tokens[0][12]}')
print_outputs(12)

position 11 wt is 23

token embedding 11
position 10 A (wt)
tensor(23)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00012
5:	 0.02222
6:	 0.89687
7:	 0.00032
8:	 0.02151
9:	 0.00015
10:	 0.00103
11:	 0.00243
12:	 0.00011
13:	 0.00022
14:	 0.00150
15:	 0.00019
16:	 0.00032
17:	 0.00082
18:	 0.00017
19:	 0.00012
20:	 0.00021
21:	 0.00016
22:	 0.00010
23:	 0.05145
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 11
position 10 masked
tensor(23)
0:	 0.00000
1:	 0.00000
2:	 0.00000
3:	 0.00000
4:	 0.00020
5:	 0.03253
6:	 0.82966
7:	 0.00065
8:	 0.03876
9:	 0.00017
10:	 0.00109
11:	 0.00466
12:	 0.00026
13:	 0.00026
14:	 0.00254
15:	 0.00020
16:	 0.00035
17:	 0.00092
18:	 0.00030
19:	 0.00018
20:	 0.00032
21:	 0.00020
22:	 0.00016
23:	 0.08659
24:	 0.00000
25:	 0.00000
26:	 0.00000
27:	 0.00000
28:	 0.00000
29:	 0.00000
30:	 0.00000
31:	 0.00000
32:	 0.00000

token embedding 11
position 10 L (mut)
t

/tmp/ipykernel_3261046/1125080389.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for nj,j in enumerate(F.softmax(output['logits'][i][t]).tolist()):


In [14]:
import numpy as np
x = np.arange(10)
np.random.choice(x, size = 10, replace = False)

array([0, 8, 5, 1, 7, 2, 6, 9, 4, 3])

# ESM 3

In [3]:
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig
from esm.models.esm3 import ESM3
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, SamplingConfig, LogitsConfig
from saveAndLoad import *

device = 'cuda:1'

refs = pickleLoad('../../aa/canonical_ref.pkl')
seq = refs[0]

def feature_select(sequence_features):
    select_feature = 'cls'

    # Apply feature selection based on select_feature
    if select_feature == 'patch':  # Default behavior for 'patch'|
        sequence_features = [features[1:] for features in sequence_features]  # Skipping the CLS token
    elif select_feature == 'cls_patch':
        sequence_features = sequence_features  # Keep the entire sequence including CLS
    elif select_feature == 'cls':  # Keep only the CLS token
        sequence_features = [features[:1] for features in sequence_features]
    else:
        raise ValueError(f'Unexpected select feature: {select_feature}')    
    return sequence_features

def encode_esm3(sequences,device = device):
    device = torch.device(device) if type(device) == str else device

    client = ESM3.from_pretrained("esmc_300m", device=device)

    protein_objects = [ESMProtein(sequence=seq) for seq in sequences]
    
    def get_features(obj):
        protein_tensor = client.encode(obj)
        output = client.forward_and_sample(
            protein_tensor, SamplingConfig(return_per_residue_embeddings=True))
        return output.per_residue_embedding
    
    sequence_features = [get_features(obj) for obj in protein_objects]
    sequence_features = feature_select(sequence_features)
    
    return sequence_features

loading data from ../../aa/canonical_ref.pkl


In [10]:
emb = encode_esm3(refs[0:2])


Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 138967.90it/s]
/home/dandreas/.conda/envs/esm3/lib/python3.10/site-packages/esm/pretrained.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to t

In [4]:
import numpy as np
from tqdm import tqdm
mut_embeddings = np.zeros((len(refs),1536),dtype=np.float32)
count = 0
for i,seq in tqdm(enumerate(refs[:5]), total = len(refs[:5])):
    if seq == '': continue
    with torch.no_grad():
        embedding = encode_esm3([seq],device = device)[0].detach().to('cpu').numpy()
        mut_embeddings[i] = embedding


# np.save('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_cls_embeddings_esm3.npy',mut_embeddings)

Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 98478.86it/s]
/home/dandreas/.conda/envs/esm3/lib/python3.10/site-packages/esm/pretrained.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to th

: 

In [2]:
from saveAndLoad import *
import numpy as np

# muts = pickleLoad('../../aa/canonical_mut.pkl')
mut2ref = pickleLoad('../../aa/idxMap_canonical_mut_to_ref.pkl')
# mut2type = pickleLoad('../../aa/mut2type.pkl')
refs = pickleLoad('../../aa/canonical_ref.pkl')
# data = pickleLoad('../../data_processing/consolidated_data.pkl')
# tumors = pickleLoad('../../aa/tumors.pkl') 
esm3_embeddings = np.load('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_cls_embeddings_esm3.npy')

with open('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_seqs.txt','r') as f:
    mut_seqs = [line.strip() for line in f]

with open('/data/dandreas/SomaticMutationsLLM/aa/canonical_ref_seqs.txt','r') as f:
    ref_seqs = [line.strip() for line in f]


mut_embeddings = np.load('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_cls_embeddings_esm3.npy')

need_indices_mut = []
for i, seq in enumerate(mut_seqs):
    if seq != '' and mut_embeddings[i].sum() == 0:
        need_indices_mut.append(i)

ref_idxs = set()
for i in need_indices_mut:
    ref_idx = mut2ref[i]
    ref = refs[ref_idx]
    if ref != '':
        ref_idxs.add(ref_idx)

print(len(ref_idxs))
ref_idxs
    

loading data from ../../aa/idxMap_canonical_mut_to_ref.pkl
loading data from ../../aa/canonical_ref.pkl
22


{69,
 152,
 208,
 505,
 661,
 804,
 805,
 817,
 1007,
 1015,
 1229,
 1305,
 1309,
 1310,
 1311,
 1312,
 1313,
 1317,
 1324,
 1332,
 1334,
 1349}

In [3]:
max = 0,0
for ni,i in enumerate(ref_seqs):
    x = len(i)
    if x>max[0]: max = x,ni
max

(14507, 1324)

In [4]:
ref_embeddings = np.load('/data/dandreas/SomaticMutationsLLM/aa/canonical_ref_cls_embeddings_esm3.npy')
need_indices_ref = []
for i, seq in enumerate(ref_seqs):
    if seq != '' and ref_embeddings[i].sum() == 0:
        need_indices_ref.append((i,len(seq)))

for i in sorted(need_indices_ref, key = lambda x: x[1]):
    print(i[0])
print(len(need_indices_ref),'\n')

152
804
1015
1310
817
1309
1305
1007
208
1229
661
1312
1349
1334
505
1311
1332
69
1317
1313
805
1324
22 



In [ ]:
all_refs = []
for r in refs:
    if (r is None) or ('*' in r):
        all_refs.append('')
    else:
        all_refs.append(r)

#save the list of all_seqs to file as line separated strings
with open('/data/dandreas/SomaticMutationsLLM/aa/canonical_ref_seqs.txt','w') as f: 
    for seq in all_refs: 
        f.write(seq+'\n')

In [7]:
from esm.models.esm3 import ESM3
from esm.sdk.api import SamplingConfig, ESMProtein
device = torch.device('cuda:0')

client = ESM3.from_pretrained("esm3_sm_open_v1", device=device)

seq = refs[1]
print('seq len', len(seq))

# protein = ESMProtein(sequence='<bos>')
protein = ESMProtein(sequence=seq)
print('protein\n',protein,'\n')

protein_tensor = client.encode(protein)
print('protein_tensor\n',protein_tensor,'\n')
output_3 = client.forward_and_sample(
    protein_tensor, SamplingConfig(return_per_residue_embeddings=True)
)

print('output\n',output_3,'\n')

/home/dandreas/.conda/envs/esm3/lib/python3.10/site-packages/esm/pretrained.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


seq len 766
protein
 ESMProtein(sequence='MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEEVWNIKQMIKLTQEHIEALLDKFGGEHNPPSIYLEAYEEYTSKLDALQQREQQLLESLGNGTDFSVSSSASMDTVTSSSSSSLSVLPSSLSVFQNPTDVARSNPKSPQKPIVRVFLPNKQRTVVPARCGVTVRDSLKKALMMRGLIPECCAVYRIQDGEKKPIGWDTDISWLTGEELHVEVLENVPLTTHNFVRKTFFTLAFCDFCRKLLFQGFRCQTCGYKFHQRCSTEVPLMCVNYDQLDLLFVSKFFEHHPIPQEEASLAETALTSGSSPSAPASDSIGPQILTSPSPSKSIPIPQPFRPADEDHRNQFGQRDRSSSAPNVHINTIEPVNIDDLIRDQGFRGDGGSTTGLSATPPASLPGSLTNVKALQKSPGPQRERKSSSSSEDRNRMKTLGRRDSSDDWEIPDGQITVGQRIGSGSFGTVYKGKWHGDVAVKMLNVTAPTPQQLQAFKNEVGVLRKTRHVNILLFMGYSTKPQLAIVTQWCEGSSLYHHLHIIETKFEMIKLIDIARQTAQGMDYLHAKSIIHRDLKSNNIFLHEDLTVKIGDFGLATVKSRWSGSHQFEQLSGSILWMAPEVIRMQDKNPYSFQSDVYAFGIVLYELMTGQLPYSNINNRDQIIFMVGRGYLSPDLSKVRSNCPKAMKRLMAECLKKKRDERPLFPQILASIELLARSLPKIHRSASEPSLNRAGFQTEDFSLYACASPKTPIQAGGYGAFPVH', secondary_structure=None, sasa=None, function_annotations=None, coordinates=None, plddt=None, ptm=None, potential_sequence_of_concern=False) 

protein_tensor
 ESMProteinTensor(sequence=tens

In [8]:
from esm.models.esmc import ESMC
from esm.sdk.api import LogitsConfig
device = torch.device('cuda:0')

client = ESMC.from_pretrained("esmc_600m", device=device)

seq = refs[1]
print('seq len', len(seq))

# protein = ESMProtein(sequence='<bos>')
protein = ESMProtein(sequence=seq)
print('protein\n',protein,'\n')

protein_tensor = client.encode(protein)
print('protein_tensor\n',protein_tensor,'\n')
output_C = client.logits(
    protein_tensor, LogitsConfig(sequence=True, return_embeddings = True)
)

print('output\n',output_C,'\n')

/home/dandreas/.conda/envs/esm3/lib/python3.10/site-packages/esm/pretrained.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


seq len 766
protein
 ESMProtein(sequence='MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEEVWNIKQMIKLTQEHIEALLDKFGGEHNPPSIYLEAYEEYTSKLDALQQREQQLLESLGNGTDFSVSSSASMDTVTSSSSSSLSVLPSSLSVFQNPTDVARSNPKSPQKPIVRVFLPNKQRTVVPARCGVTVRDSLKKALMMRGLIPECCAVYRIQDGEKKPIGWDTDISWLTGEELHVEVLENVPLTTHNFVRKTFFTLAFCDFCRKLLFQGFRCQTCGYKFHQRCSTEVPLMCVNYDQLDLLFVSKFFEHHPIPQEEASLAETALTSGSSPSAPASDSIGPQILTSPSPSKSIPIPQPFRPADEDHRNQFGQRDRSSSAPNVHINTIEPVNIDDLIRDQGFRGDGGSTTGLSATPPASLPGSLTNVKALQKSPGPQRERKSSSSSEDRNRMKTLGRRDSSDDWEIPDGQITVGQRIGSGSFGTVYKGKWHGDVAVKMLNVTAPTPQQLQAFKNEVGVLRKTRHVNILLFMGYSTKPQLAIVTQWCEGSSLYHHLHIIETKFEMIKLIDIARQTAQGMDYLHAKSIIHRDLKSNNIFLHEDLTVKIGDFGLATVKSRWSGSHQFEQLSGSILWMAPEVIRMQDKNPYSFQSDVYAFGIVLYELMTGQLPYSNINNRDQIIFMVGRGYLSPDLSKVRSNCPKAMKRLMAECLKKKRDERPLFPQILASIELLARSLPKIHRSASEPSLNRAGFQTEDFSLYACASPKTPIQAGGYGAFPVH', secondary_structure=None, sasa=None, function_annotations=None, coordinates=None, plddt=None, ptm=None, potential_sequence_of_concern=False) 

protein_tensor
 ESMProteinTensor(sequence=tens

In [16]:
output_C.embeddings.shape

torch.Size([1, 768, 1152])

In [17]:
output_3.per_residue_embedding.shape

torch.Size([768, 1536])